# Contract risk analysis

In [4]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.14"
#r "nuget: Scriban"
#r "nuget: dotenv.net"

using Azure;
using Azure.AI.OpenAI;
using Scriban;

using dotenv.net;
DotEnv.Load();

var endpoint=Environment.GetEnvironmentVariable("OPENAI_URI");
var key=Environment.GetEnvironmentVariable("OPENAI_KEY");
var model=Environment.GetEnvironmentVariable("OPENAI_GPT_DEPLOYMENT");


Installed Packages Azure.AI.OpenAI, 1.0.0-beta.14 dotenv.net, 3.1.3 Scriban, 5.9.1

### Process a completion using the Azure SDK

In [5]:
async Task<string> ProcessCompletion(string prompt, int max_tokens=500, float temperature=0.3f)
{
    OpenAIClient client = new(new Uri(endpoint), new AzureKeyCredential(key));

    var chatCompletionsOptions = new ChatCompletionsOptions()
    {
        DeploymentName = model, //This must match the custom deployment name you chose for your model
        Messages =
        {
            new ChatRequestAssistantMessage(prompt),
        },
        MaxTokens = max_tokens,
        Temperature = temperature
    };

    Response<ChatCompletions> response = await client.GetChatCompletionsAsync(chatCompletionsOptions);

    return response.Value.Choices[0].Message.Content;
}

### Mock getting a contract

In [6]:
string GetLeaseAgreemenet(string title) {
    if (title=="LEASE_AGREEMENT")
    return @"RESIDENTIAL LEASE AGREEMENT
RENT. The Tenant shall pay to Landlord the sum of $1,500 per month (hereinafter referred to as ""Rent"") for the duration of the Term of the Lease. The Rent shall be payable on or before every day of the month (hereinafter referred to as the ""Due Date""), notwithstanding that the said date falls on a weekend or holiday.
A. Late Rent. If Rent is not paid within days of the Due Date, the Rent shall be considered past due, and a late fee of a $50 or 5% of the Rent past due shall be applied for every day Rent is late or O occurrence Rent is late.
B. Returned Checks. In the event that a check intended as payment for Rent is dishonored for whatever reason, the same shall be considered as Late Rent with the late fee being payable on the same.";
    else
    return string.Empty;
}

### Render a template

In [7]:
string RenderTemplate(string template, dynamic data) {
    var templateObject = Template.Parse(template);
    return templateObject.Render(data);
}

### Define the template

In [8]:
var input = GetLeaseAgreemenet("LEASE_AGREEMENT");

var analysis_template = @"System:
You are an agent that can help summarize and analyze contracts for risk. Be professional and courteous.

User:
For the following text, summarize and list risks.
{{input}}

Output format:
Summary: 
""

Risks: 
-||-
";

### Process and print the results

In [9]:
Console.WriteLine(await ProcessCompletion(RenderTemplate(analysis_template, new { input = input })));

Summary:
The Residential Lease Agreement stipulates that the Tenant is required to pay the Landlord $1,500 monthly as Rent on or before the Due Date each month. If Rent is not paid within the specified days after the Due Date, it is considered past due, and a late fee will be charged daily or per occurrence. Additionally, any dishonored checks will incur the same late fees as Late Rent.

Risks:
- Risk of incurring late fees if Rent is not paid on time, which could add a significant amount to the monthly payment.
- Risk of additional charges if payment is made by check and the check is dishonored.
- Potential for accumulating daily late fees if Rent remains unpaid, increasing the financial burden on the Tenant.
